# Trabalho Prático 3

## Introdução

A resolução deste trabalho prático tem 3 objetivos principais:

- Criar uma classe **Python** que implemente o algoritmo de _Boneh & Venkatesan_ .
- Implementação de um esquema de assinaturas digitais _FALCON_ .
- Estudar gamas de valores de determinados parâmetros que tornem viável um ataque de inversão de chave pública ou inversão do criptograma utilizando redução de bases.

Este relatório está, desta forma, dividido em três partes, cada parte correspondente à resolução de um dos problemas e, além disso, está estruturado de forma a que o texto entre os _snippets_ de código seja suficientemente explicativo sobre a implementação e desenho da solução em cada um dos problemas.

## Algoritmo de Boneh & Venkatesan

Esta secção tem como propósito definir uma classe **Python** que implemente o algoritmo de _Boneh & Venkatesan_ , que explora o _Hidden Number Problem_ que, se bem sucedido, permite obter um segredo a partir de um conjunto de dados. Além disso, a secção está dividida em quatro partes:

- A primeira parte define o oráculo HNP, que gera um segredo,permite calcular o msb e compara um segredo calculado com o gerado;
- A segunda parte define as funções de geração do vetor aleatório **x** e do vetor composto por $ ui = msb(xi,s) $ para $ i=0 $ até $ i = l - 1 $.
- A terceira parte define apenas o algoritmo de _Boneh & Venkatesan_ .
- A quarta parte, apelidada de teste, fornece os parâmetros necessários à instância do algoritmo para que ele descubra o segredo **s** a partir dos mesmos.

### Definição do oráculo HNP

O propósito desta secção passa por definir o oráculo que gera um segredo e, posteriormente, retorna o vetor u, com l elementos, onde cada um é um inteiro representativo dos k bits mais significativos de $ s * xi $.

In [52]:

class HNPOracle:
    
    def __init__(self,p):
        # gerar o segredo.
        self.secret = ZZ.random_element(p)
        print 'secret'
        print self.secret
    
    def msb(self,k,p,xi):
        value = ZZ(Mod(xi*self.secret,p))
        binary_value = value.digits(2)
        binary_value.reverse()
        value_str = ''
        if len(binary_value) < k:
            for i in range(0,len(binary_value)):
                value_str += str(binary_value[i])
        else:
            for i in range(0,k):
                value_str += str(binary_value[i])
        return value_str
    
    def compare_secret(self,calculated_secret):
        if calculated_secret == self.secret:
            return True
        else:
            return False

### Definição da função de geração do vetor aletório e vetor u

In [53]:
def generate_l_random_elements(l,p):
    x = []
    for i in range(0,l):
        x.append(ZZ.random_element(p))
    return x

def calculate_u_vector(x_vector,k,p,oracle):
    u_vector = []
    for xi in x_vector:
        ui = oracle.msb(k,p,xi)
        ui = ZZ(int(ui,2))
        u_vector.append(ui)
    return u_vector  

### Definição do algoritmo de Boneh & Venkatesan

In [54]:
import sage.modules.free_module_integer as fmi
import numpy as np

class BV:
    
    def __init__(self,u,x,k,p,l):
        # construir a matriz L , lambda,o target T e, finalmente, o reticulado Lret
        self.param_lambda = 2^(k+1)
        self.L = self.param_lambda * p * matrix.identity(l)
        self.L = self.L.transpose()
        self.L = self.L.insert_row(l,zero_vector(l))
        self.L = self.L.transpose()
        temp_x = [self.param_lambda * i for i in x]
        temp_x.append(1)
        self.L = self.L.insert_row(l,temp_x)
        self.target = [self.param_lambda * i for i in u]
        self.target.append(0)
        self.target = matrix(self.target)
        self.Lret = fmi.IntegerLattice(self.L)
        
    
    def solve(self,x,p,l):
        # Calcular o CVP aproximado do reticulado Lret
        L = matrix(self.Lret.reduced_basis)
        t = matrix(1,l+1,list(-self.target))
        zero = matrix(l+1,1,[0]*(l+1))
        M = matrix(1,1,p**2)
        L1 = block_matrix(2,2,[[L,zero],[t,M]])
        ret = fmi.IntegerLattice(L1).reduced_basis
        error1 = np.array(ret[l+1][:-1])
        y1 = error1 + self.target
        return y1[0][l] # última componente do vetor resultante.
        

### Teste ao algoritmo de Boneh & Venkatesan

In [78]:
l = 2^7
k = 64
p = 2^64
x_vector = generate_l_random_elements(l,p)
oracle = HNPOracle(p)
u_vector = calculate_u_vector(x_vector,k,p,oracle)
bv = BV(u_vector,x_vector,k,p,l)
calculated_secret = bv.solve(x_vector,p,l)
print 'calculated_secret'
print calculated_secret
if oracle.compare_secret(calculated_secret):
    print 'Algoritmo aproximado calculou o segredo com sucesso!'
else:
    print 'Algoritmo aproximado não conseguiu calcular o segredo com sucesso!'

secret
6981224216842594255
calculated_secret
6981224216842594255
Algoritmo aproximado calculou o segredo com sucesso!


## Definição do Esquema NTRU-Encrypt

In [237]:
import random
import hashlib
from sage.crypto.util import ascii_to_bin, bin_to_ascii

name        = "NTRU_PKE_443"
d           = 115
N           = 443
p           = 3
q           = next_prime(p*N)       
max_msg_len = 33 
Z.<x>       = ZZ[]
Q.<x>       = PolynomialRing(GF(q),name='x').quotient(x^N-1)

def pad(msg):
    #msg = ascii_to_bin(msg); msg = list(msg)
    
    msg_len = len(msg)/8
    if msg_len > max_msg_len:
        raise Exception('msg_len should not exceed {}. The value of msg_len was: {}'
                        .format(max_msg_len, msg_len))
        
    r = N - (167 + 6 + msg_len*8)
    lr = list(0 for i in (0..r-1))
    
    msg_len = "{0:06b}".format(int(msg_len))
    msg_len = [int(d) for d in msg_len[:6]];
        
    m = msg + lr + vec(167) + msg_len;
        
    return m

def hash_message(m, h):
    m = ''.join([str(x) for x in m])
    hm = hashlib.sha512(m)
    lh = map(lift,h.list())
    sh = ''.join(str(x) for x in lh)
    hh = hashlib.sha512(sh)
    rseed = str(hm) + str(hh)
    return rseed

def vec(n):
    return  [choice([-1,0,1]) for k in range(n)]

# arredondamento módulo 'q'
def qrnd(f):
    qq = (q-1)//2 ; ll = map(lift,f.list())
    return [n if n <= qq else n - q  for n in ll]

# arredondamento módulo 'p'
def prnd(l):
    pp = (p-1)//2
    rr = lambda x: x if x <= pp else x - p        
    return [rr(n%p) if n>=0 else -rr((-n)%p) for n in l]

def extract(m):
    msg_len = m[-6:]
    msg_len = ''.join(str(x) for x in msg_len)
    msg_len = int(msg_len,2)
    
    msg = m[:msg_len*8]
    
    #msg = ''.join(str(x) for x in msg); msg = bin_to_ascii(msg)
    
    return msg, msg_len

class NTRU:
    
    def __init__(self):
        print('Name:\t' + name)
        print('d:\t' + str(d))
        print('N:\t' + str(N))
        print('p:\t' + str(p))
        print('q:\t' + str(q))
        print Z
        print Q
    
    def keypair(self):
        f = Q(0)
        while not f.is_unit():
            F = Q(vec(N)); f = 1 + p*F
        G = Q(vec(N)) ; g = p*G
        self.f = f
        self.h = f^(-1) * g
    
    def encrypt(self, msg):
        m = pad(msg)
        
        rseed = hash_message(m, self.h)
        random.seed(rseed)        
        r = [random.choice([-1,0,1]) for k in range(N)]
        
        t = Q(r) * self.h

        lt = map(lift,t.list())
        st = ''.join(str(x) for x in lt)
        tseed = hashlib.sha512(st)
        random.seed(tseed)
        
        m_mask = [random.choice([-1,0,1]) for k in range(N)]
        mm = prnd(qrnd(Q(m) - Q(m_mask)))
        
        c = t + Q(mm)
        
        return c
    
    def decrypt(self,e):
        mm = prnd(qrnd(self.f * e))
        t = e - Q(mm)
        
        lt = map(lift,t.list())
        st = ''.join(str(x) for x in lt)
        tseed = hashlib.sha512(st)
        
        random.seed(tseed)
        m_mask = [random.choice([-1,0,1]) for k in range(N)]
        
        m = prnd(qrnd(Q(mm) + Q(m_mask)))
        
        rseed = hash_message(m, self.h)
        random.seed(rseed)
        r = [random.choice([-1,0,1]) for k in range(N)]
        
        msg,msg_len = extract(m)
        
        return msg

### Teste ao esquema NTRU

In [239]:
K = NTRU()

#msg = "Estruturas Criptográficas - TP3"
msg = vec(33 * 8)
print('\nmessage:\t' + str(msg))
print('message length:\t' + str(len(msg)) + ' bytes' + '\n')

K.keypair()
e = K.encrypt(msg)
print msg == K.decrypt(e)


Name:	NTRU_PKE_443
d:	115
N:	443
p:	3
q:	1361
Univariate Polynomial Ring in x over Integer Ring
Univariate Quotient Polynomial Ring in x over Finite Field of size 1361 with modulus x^443 + 1360

message:	[-1, -1, -1, 0, -1, 1, 1, 1, -1, 1, 1, 0, 0, -1, -1, -1, 1, 0, -1, 0, 0, 0, 0, 1, -1, 0, -1, 1, 0, 1, -1, -1, 1, -1, 1, -1, 0, 1, 1, 0, -1, 1, -1, 0, -1, 0, 1, -1, -1, -1, 0, 0, 1, -1, 0, -1, -1, 1, 1, 0, 1, -1, 1, 1, -1, 0, -1, -1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, -1, 0, 0, 0, 0, 0, 1, 1, -1, -1, -1, -1, -1, 1, 1, 0, 1, -1, 1, -1, -1, 0, 1, 0, 1, 1, 1, 0, -1, 1, -1, 0, 1, 1, 0, 1, 0, -1, 0, 1, -1, -1, -1, -1, 1, 0, 1, 1, 0, 0, 0, 1, -1, 1, -1, -1, 1, 1, 1, 0, 0, -1, -1, -1, 0, 0, 0, 0, 0, -1, -1, -1, 1, 0, -1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 0, 1, 0, 0, -1, 1, -1, -1, 0, 0, 0, -1, -1, 1, -1, -1, -1, 1, 0, 0, -1, -1, 1, 0, 0, -1, 1, 1, -1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, -1, 0, -1, 0, 1, 0, 1, -1, 1, 1, -1, -1, 0, -1, -1, 1, 0, 0, 0, 0, 1, 0, -1, 0, 0, 0, -1, 0, 1, -1, 1, 0, 0, 1, 1

## Ataques de inversão

### Definição da classe NTRU e da classe LAT

In [135]:
import sage.modules.free_module_integer as fmi
# http://doc.sagemath.org/html/en/reference/modules/sage/modules/free_module_integer.html

d = 6

N = 21
p = 3
q = next_prime(p*N)

Z.<x>  = ZZ[]        # polinómios de coeficientes inteiros
Q.<x>  = PolynomialRing(GF(q),name='x').quotient(x^N-1)

def vec():
    return  [choice([-1,0,1]) for k in range(N)]

# arredondamento módulo 'q'
def qrnd(f):    # argumento em 'Q'
    qq = (q-1)//2 ; ll = map(lift,f.list())
    return [n if n <= qq else n - q  for n in ll]

# arredondamento módulo 'p'
def prnd(l):
    pp = (p-1)//2
    rr = lambda x: x if x <= pp else x - p        
    return [rr(n%p) if n>=0 else -rr((-n)%p) for n in l]

class NTRU(object):
    def __init__(self):
        # calcular um 'f' invertível
        f = Q(0)
        while not f.is_unit():
            F = Q(vec()); f = 1 + p*F
        # gerar as chaves
        G = Q(vec()) ; g = p*G
        self.f = f
        self.h = f^(-1) * g
        
    def encrypt(self,m):
        r = Q(vec()) 
        return r*self.h + Q(m)

    def decrypt(self,e):
        a = e*self.f
        return prnd(qrnd(a))

class Lat(NTRU):
    def __init__(self):
        super(Lat,self).__init__()
        B1 = identity_matrix(ZZ,N); Bq = q*B1; B0 = matrix(ZZ,N,N,[0]*(N^2))
        h = qrnd(self.h)
        # rodar um vetor
        H = [h]
        for k in range(N-1):
            h = [h[-1]] + h[:-1]   # shift right rotate
            H = H + [h]
        H = matrix(ZZ,N,N,H)
        self.L = fmi.IntegerLattice(block_matrix([[Bq,B0],[H,B1]])) 


### Inversão da chave pública

Segundo o artigo de _Silverman_, encontrar a chave privada f a partir da chave pública h, é equivalente a encontrar um vetor curto no reticulado **L(h)** definido em cima, para parâmetros apropriados.

In [136]:
import numpy as np

l = Lat()
lredmat = l.L.reduced_basis.LLL()
lred = fmi.IntegerLattice(lredmat)

short_aproximate = np.array(lredmat[0][:-1]) #SVP aproximado
print Q(prnd(short_aproximate))
print l.f


66*x^20 + x^18 + 66*x^15 + 66*x^14 + 66*x^13 + 66*x^11 + 66*x^10 + x^9 + x^8 + 66*x^7 + 66*x^6 + 2*x^5 + 66*x^3 + 2*x^2 + 66*x + 66
3*x^20 + 3*x^15 + 64*x^14 + 64*x^12 + 64*x^11 + 64*x^7 + 3*x^6 + 64*x^5 + 64*x^4 + 3*x^2 + 4


### Inversão do criptograma

Segundo o artigo de _Silverman_, recuperar a mensagem original a partir do criptograma e da chave pública, é equivalente a encontrar o vetor mais próximo do target $ [0,criptograma] $, no reticulado **L(h)**.

In [174]:
import numpy as np

l = Lat()
message = vec()
print 'message:'
print Q(message)
e = l.encrypt(message)
e = qrnd(e)
vector = [0 for i in range(0,N)]
for i in e:
    vector.append(i)
zero_41_vector =  [0 for i in range(0,41)]
zero_41_vector.append(2**q)
lred = l.L.reduced_basis
lred = lred.transpose()
lred = lred.insert_row(42,zero_41_vector)
lred = lred.transpose()
L1 = fmi.IntegerLattice(lred)
lred = L1.reduced_basis

err1 = np.array(lred[41][:-1])
y1 = err1 + vector
# y1 deverá ser igual ao vetor [-r,m], retirámos -r e ficámos apenas com m.
new_vec = []
for i in range(22,42):
    new_vec.append(y1[i])
print ''
print 'calculated_message'
print Q(prnd(new_vec))

message:
66*x^19 + 66*x^18 + 66*x^17 + 66*x^15 + 66*x^14 + x^12 + 66*x^11 + x^10 + 66*x^9 + x^7 + 66*x^6 + x^3 + x^2 + x + 1

calculated_message
66*x^19 + x^18 + x^15 + 66*x^14 + x^13 + 66*x^12 + 66*x^11 + x^10 + x^9 + x^6 + x^4 + x^3 + x^2 + 66*x + 1


## Conclusão

Os resultados da realização deste trabalho prático não são, desta vez, tão satisfatórios como nos trabalhos anteriores visto que, apesar de termos conseguido implementar o algoritmo de **Boneh & Venkatesan** e o esquema **NTRUEncrypt**, não conseguimos implementar os ataques de inversão referidos no enunciado. Como é dito em cima, conseguimos entender a relação entre o cálculo do vector mais curto e o cálculo do vetor mais próximo com as inversões da chave pública e criptograma, respetivamente, mas não conseguimos implementar de forma a que conseguíssemos obter esses mesmos resultados.

Como deve ser óbvio nesta altura, este trabalho apresentou imensas dificuldades, especialmente e decididamente na pergunta 3, mas também no esquema **NTRUEncrypt**, apesar de que, nesse caso, ainda o conseguimos implementar de forma a que consiga cifrar e decifrar uma mensagem.

## Referências

1. [Worksheets TP4 do professor](https://www.dropbox.com/sh/f0j9adiaw4v3deb/AADiMJL2SBP8IMjAxA-SxX2Za/WorkSheets/TP4?dl=0&subfolder_nav_tracking=1)
2. [NTRU and Lattice-Based Crypto: Past, Present, and Future de Joseph H. Silverman](http://archive.dimacs.rutgers.edu/Workshops/Post-Quantum/Slides/Silverman.pdf)
3. [NTRUEncrypt Supporting Documentation](https://www.dropbox.com/sh/ejlraszbb4ogbod/AABacbwfTbUKwRPmPfkgEIuIa/NTRUEncrypt/Supporting_Documentation?dl=0&subfolder_nav_tracking=1)
4. [HNP e abordagem de Boneh & Venkatesan](https://paper.dropbox.com/doc/Hidden-Number-Problem-HXjSmxuD62Xr6nEXocfjg)